## Preprocessing

In [13]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
import keras_tuner as kt
from kerastuner.tuners import Hyperband
from kerastuner import HyperParameters


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


Saving the column NAME

In [14]:
# Drop the non-beneficial ID column, 'EIN'.
application_df.drop(columns= ['EIN'], axis=1, inplace=True)
application_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [15]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [16]:
# Look at APPLICATION_TYPE value counts for binning
app_type_count = application_df['APPLICATION_TYPE'].value_counts()
app_type_count

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [17]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_count[app_type_count < 500].index)


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [18]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count.head(15)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: count, dtype: int64

In [19]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_greater1 = classification_count.loc[classification_count > 1]
classification_greater1.head(10)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
Name: count, dtype: int64

Cutoff value changed. < 1000

In [20]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_count[classification_count < 1000].index)


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

Column NAME reintroduced.

In [21]:
# Look at NAME value counts for binning
name_counts = application_df['NAME'].value_counts()
name_counts

NAME
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: count, Length: 19568, dtype: int64

In [22]:
# Choose a cutoff value and create a list of names to be replaced
names_to_replace = list(name_counts[name_counts < 200].index)

# Replace in dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(
        name, "Other")

# Check to make sure binning was successful
application_df['NAME'].value_counts()

NAME
Other                                                                 28123
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                         408
PTA TEXAS CONGRESS                                                      368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                           331
ALPHA PHI SIGMA                                                         313
TOASTMASTERS INTERNATIONAL                                              293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                       287
LITTLE LEAGUE BASEBALL INC                                              277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                266
MOMS CL

In [23]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_data = pd.get_dummies(application_df)
numeric_data = numeric_data.astype(int)
numeric_data.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_INTERNATIONAL ASSOCIATION OF LIONS CLUBS,NAME_INTERNATIONAL ASSOCIATION OF SHEET METAL AIR RAIL & TRANSPORTATION,NAME_LITTLE LEAGUE BASEBALL INC,NAME_MOMS CLUB,NAME_MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [24]:
# Split our preprocessed data into our features and target arrays
y = numeric_data ['IS_SUCCESSFUL']
X = numeric_data.drop(columns='IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=78)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

Optimization.
Adding a thir layer.
Activation changed

In [33]:

def create_model(hp, input_features):
    nn_model3 = tf.keras.Sequential([
        # Specify the input shape in the Input layer
        tf.keras.layers.Input(shape=(input_features,)),
    ])

    # Allow Keras Tuner to decide which activation function to use in hidden layers
    activation_choice = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])
    
    # Allow Keras Tuner to decide the number of neurons in the first layer
    nn_model3.add(tf.keras.layers.Dense(
        units=hp.Int('first_units', min_value=1, max_value=90, step=5),
        activation=activation_choice))

    # Allow Keras Tuner to decide the number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model3.add(tf.keras.layers.Dense(
            units=hp.Int(f'units_layer_{i}', min_value=1, max_value=30, step=5),
            activation=activation_choice))
    
    nn_model3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Allow Keras Tuner to decide the learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    # Compile the model
    nn_model3.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"])
    
    return nn_model3


Number of epoch increased.

In [34]:

input_features = X_train_scaled.shape[1]

def build_model(hp):
    return create_model(hp, input_features)

# Initialize the tuner
tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=25,
    factor=3,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)


In [35]:
tuner.search(X_train_scaled, y_train, epochs=25, validation_split=0.2)


Trial 30 Complete [00h 00m 09s]
val_accuracy: 0.7473275065422058

Best val_accuracy So Far: 0.7484936714172363
Total elapsed time: 00h 01m 48s


In [36]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train it on the data for 25 epochs
fit_model = model.fit(X_train_scaled, y_train, epochs=25)


Epoch 1/25
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 404us/step - accuracy: 0.7211 - loss: 0.5558
Epoch 2/25
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - accuracy: 0.7478 - loss: 0.5144
Epoch 3/25
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 450us/step - accuracy: 0.7530 - loss: 0.5052
Epoch 4/25
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step - accuracy: 0.7537 - loss: 0.5024
Epoch 5/25
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 402us/step - accuracy: 0.7585 - loss: 0.4974
Epoch 6/25
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step - accuracy: 0.7564 - loss: 0.4976
Epoch 7/25
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step - accuracy: 0.7531 - loss: 0.5018
Epoch 8/25
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - accuracy: 0.7576 - loss: 0.5000
Epoch 9/25
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - accuracy: 0.7580 - loss: 0.4976
Epoch 10/25
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 426us/step - accuracy: 0.7615 - loss: 0.4955
Epoch 11/25
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step - accuracy: 0.7553 - loss: 0.4993
Epoch 12/25
804/804 ━━━━━━━━━━

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)

# Print the loss and accuracy
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - 548us/step - accuracy: 0.7482 - loss: 0.5079
Loss: 0.5079114437103271, Accuracy: 0.7482215762138367


A loss value of 50 indicates that the model works better than the 2 previous models.

The accuracy percent shows that 74% of the model's predicted values align with the true values in the original dataset.

In [39]:
#nn_model3.save('Models/AlphabetSoupCharityOptimization.h5')
#Warning: this file format is considered legacy. We recommend using instead the native Keras format,

In [42]:
# Save the model using the native Keras format

model.save('Models/AlphabetSoupCharityOptimization2.keras')




A loss value of 51 indicates that the model works better, but can be still further optimized.

The accuracy percent shows that 74% of the model's predicted values align with the true values in the original dataset.